In [1]:
import pandas as pd
import numpy as np
import pyfaidx
import regex
from tqdm.autonotebook import tqdm

/Users/milessmith/miniconda3/envs/mc/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [28]:
spacers = pd.read_pickle('spacers_df.pkl')

In [29]:
spacers.head()

,hash,spacer,off_target_score,mismatches
0,-1245412300131519133,CTCCCCTTTCGGACAGCTC,15.839646,395
1,-7107283595214827167,TCCCCTTTCGGACAGCTCA,21.880718,329
2,-5470846543948195797,ACCGAGAATCGAAACTAAG,36.057828,170
3,3044478046141746202,CCGAGAATCGAAACTAAGC,55.348827,102
4,-693125485272288290,CGAGAATCGAAACTAAGCT,40.496279,141


In [30]:
spacers["on_target_score"] = np.repeat(100, spacers.shape[0])
spacers["gene_name"] = np.repeat("PML", spacers.shape[0])

In [31]:
on_target_score_threshold = 50
off_target_score_threshold = 0

In [32]:
spacers = spacers[spacers["on_target_score"] > on_target_score_threshold]
spacers = spacers[spacers["off_target_score"] > off_target_score_threshold]

In [34]:
spacers.shape

(1203, 6)

In [36]:
spacers_per_feature = 6
tqdm.pandas(desc="Assembling library", unit="spacers")
grouped = (
    spacers.groupby("gene_name")
    .progress_apply(
        lambda x: x.nlargest(spacers_per_feature, "on_target_score")
    )
    .reset_index(drop=True)
)
grouped

,hash,spacer,off_target_score,mismatches,on_target_score,gene_name
0,-1245412300131519133,CTCCCCTTTCGGACAGCTC,15.839646,395,100,PML
1,-7107283595214827167,TCCCCTTTCGGACAGCTCA,21.880718,329,100,PML
2,-5470846543948195797,ACCGAGAATCGAAACTAAG,36.057828,170,100,PML
3,3044478046141746202,CCGAGAATCGAAACTAAGC,55.348827,102,100,PML
4,-693125485272288290,CGAGAATCGAAACTAAGCT,40.496279,141,100,PML
5,585601195684176837,CGAAACTAAGCTGGGGTCC,30.793663,205,100,PML


In [38]:
len(spacers['spacer'])

1203

In [39]:
len(spacers['spacer'].unique())

414

In [42]:
spacers.groupby('spacer').first().reset_index()

,spacer,hash,off_target_score,mismatches,on_target_score,gene_name
0,AAACCTGAGTGACTAGGCT,7946757728487399660,20.375308,480,100,PML
1,AAACTTCTTCCGGGCCCTG,-6550736683021627494,24.059597,427,100,PML
2,AAAGGGGAGGTGCCGGCTC,2041551058632606271,31.289748,427,100,PML
3,AAATGACACGCCAGGAGCC,2630714792824120367,20.651063,465,100,PML
4,AACCACAGGTCTGGGGTAC,-6771401189030078906,23.692743,477,100,PML
5,AACCACGTGGCCAGTGGCG,-6362365875919671034,37.998011,276,100,PML
6,AACTGCCCCTCATGAGGTT,-1159460600066611991,25.116063,461,100,PML
7,AACTGCCTAGGATGCATGC,-388829239190398224,23.807597,406,100,PML
8,AAGCCAGCAAATGACACGC,-8062130682372017589,27.139731,464,100,PML
9,AAGGTTGGCTCGGAGCTCC,-285043384876935667,23.952400,329,100,PML
